In [3]:
#import library
import numpy as py
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM , SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re

In [5]:
#read and load data
data = pd.read_csv('/content/Sentiment.csv')
#select important features
data = data[['text','sentiment']]

In [7]:
#expcetion neutral case from lable sentiment
data = data[data.sentiment != 'Neutral']
#lambda func:- create function without name
#where x is parametar, x.lower is the return
#data['text'].apply represent call by value to x
data['text']= data['text'].apply(lambda x: x.lower()) # transformation upper case to lower
data['text']= data['text'].apply(lambda x:re.sub(r'[^a-zA-Z0-9\s]','',x)) #Determine the things I want in the speech

In [8]:
for idx, row in data.iterrows():
  row[0] = row[0].replace('rt',' ') # replace rt with a space

In [10]:
max_features=2000 # the variable is represent dictionary

#tokenizer is a class Responsible for dividing the sentence, as it divides every space
#create object from tokenizer class and give it two variables, one is the size of the dictionary and the other is the thing I want to divide from
tokenizer= Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
x = tokenizer.texts_to_sequences(data['text'].values)

x= pad_sequences(x) #To make all sentences equal, where each sentence will be equal to the longest sentence in text label

In [11]:
embed_dim =128 ## represent numbers of weights
lstm_out=196 ## represent numbers of units

model= Sequential()
#Embedding is the first layer in the model Responsible for give Giving words that are close or that have a relationship between them similar values
#max_features --> dictionary
#embed_dim --> numbers of weights
#input_length --> It represents the input of the layer
model.add(Embedding(max_features, embed_dim,input_length=x.shape[1]))

#drop from output of the Embedding layers to  prevent happening overfitting
model.add(SpatialDropout1D(0.4))

#lstm layers tekes 3 variables
#lstm_out --> numbers of units
#dropout --> represent a random drop to the input
#recurrent_dropout--> represent a random drop to the recurrent
model.add(LSTM(lstm_out, dropout=0.2 , recurrent_dropout= 0.2))

model.add(Dense(2,'softmax'))
model.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

y=pd.get_dummies(data['sentiment']).values ## one hot incoding

In [12]:
x_train, x_test ,y_train, y_test= train_test_split(x,y, test_size=0.2,random_state=32)

In [13]:
model.fit(x_train, y_train, epochs=5 , batch_size=30)

Epoch 1/5
287/287 [==============================] - 47s 151ms/step - loss: 0.4223 - accuracy: 0.8217
Epoch 2/5
287/287 [==============================] - 49s 170ms/step - loss: 0.3190 - accuracy: 0.8635
Epoch 3/5
287/287 [==============================] - 50s 175ms/step - loss: 0.2766 - accuracy: 0.8829
Epoch 4/5
287/287 [==============================] - 44s 154ms/step - loss: 0.2477 - accuracy: 0.8981
Epoch 5/5
287/287 [==============================] - 43s 151ms/step - loss: 0.2224 - accuracy: 0.9075
